In [16]:
import os
import pandas as pd
import yfinance as yf
from fredapi import Fred

fred = Fred(api_key="aa4422c1ac7545c5837f4e5b7b52d6c8")
fred_indicators = {
    "10Y_Treasury_Rate": "DGS10",
    "Fed_Funds_Rate": "FEDFUNDS",
    "Inflation_CPI": "CPIAUCSL",
    "High_Yield_Spread": "BAMLH0A0HYM2"
}
etf_proxies = {
    "10Y_Treasury_ETF_IEF": "IEF",
    "Fed_Funds_Proxy_SHV": "SHV",
    "Inflation_Protected_TIP": "TIP",
    "High_Yield_Bonds_HYG": "HYG"
}

periods = {
    "1M Change %": 22,
    "3M Change %": 66,
    "6M Change %": 132,
    "1Y Change %": 264
}

def compute_change(series, days):
    series = pd.to_numeric(series, errors="coerce").dropna()
    if len(series) < days:
        return None
    return round(((series.iloc[-1] - series.iloc[-days]) / series.iloc[-days]) * 100, 2)

fred_data = []

for name, code in fred_indicators.items():
    print(f"-> Fetching FRED: {name}")
    series = fred.get_series(code)
    row = {"Indicator": name, "Code": code}
    for label, days in periods.items():
        row[label] = compute_change(series, days)
    fred_data.append(row)

fred_df = pd.DataFrame(fred_data)
fred_df.to_csv("market-data/bonds-market-data.csv", index=False)
print("-> Saved: market-data/bonds-market-data.csv")

os.makedirs("historical-data/bonds", exist_ok=True)

for name, ticker in etf_proxies.items():
    print(f"-> Downloading OHLC for ETF: {name} ({ticker})")
    df = yf.download(ticker, period="1y", interval="1d")
    df.reset_index(inplace=True)
    df.to_csv(f"historical-data/bonds/{name}.csv", index=False)

-> Fetching FRED: 10Y_Treasury_Rate
-> Fetching FRED: Fed_Funds_Rate
-> Fetching FRED: Inflation_CPI
-> Fetching FRED: High_Yield_Spread
-> Saved: market-data/bonds-market-data.csv
-> Downloading OHLC for ETF: 10Y_Treasury_ETF_IEF (IEF)


[*********************100%***********************]  1 of 1 completed


-> Downloading OHLC for ETF: Fed_Funds_Proxy_SHV (SHV)


[*********************100%***********************]  1 of 1 completed


-> Downloading OHLC for ETF: Inflation_Protected_TIP (TIP)


[*********************100%***********************]  1 of 1 completed


-> Downloading OHLC for ETF: High_Yield_Bonds_HYG (HYG)


[*********************100%***********************]  1 of 1 completed
